- Read target text file
- create dict of words seen
- read wordspace file
    - parse line
    - parse sparse vector
    - retain only if token is in dict
- randomise vec for other words
- build cky parser or similar for utterances
- go thru utterances


In [1]:
import re
import random
import math
import time

In [2]:
wordspacefile="/home/jussi/Desktop/data/wordspaces/gavagai-news-22.wordspace"
lexiconfile="/home/jussi/Desktop/data/EventStatus/words.txt"
textfile="/home/jussi/Desktop/data/EventStatus/chunks.txt"

In [3]:
def dolexicon():
    lexicon={}
    lexlinepattern=re.compile(r'^(\d+)\s+([\-\w]+)\s+([\.\d]+)$')
    lexfile=open(lexiconfile,'r')
    lexline=lexfile.readline().rstrip()
    while lexline:
        m=lexlinepattern.match(lexline)
        if m:
            word=m.groups()[1]
            idf=m.groups()[2]
            lexicon[word]=idf
#        else:
#            print("no match for ",lexline)
        lexline=lexfile.readline().rstrip()            
    lexfile.close()
    return lexicon

In [54]:
def sparseadd(onevec,othvec,weight):
    result={}
    if onevec:
        for l in onevec:
            result[l] = onevec[l]
    for k in othvec:
        if k in result:
            result[k] = result[k]+othvec[k]*float(weight)
        else:
            result[k] = othvec[k]*float(weight)
    return result

def sparsecosine(xvec,yvec):
    x2 = 0;
    y2 = 0;
    xy = 0;
    try:
        for i in range(len(xvec)):
            if i in xvec:
                x2 += xvec[i]*xvec[i]
            if i in yvec:
                y2 += yvec[i]*yvec[i]
                if i in xvec:
                    xy += xvec[i]*yvec[i]
    except:
        print("error at position ",i)
    if (x2+y2 == 0):
        cos = 0
    else:
        cos = xy/(math.sqrt(x2)+math.sqrt(y2))
    return cos

def lengthweightedsparsecosine(xvec,xl,yvec,yl):
    x2 = 0;
    y2 = 0;
    xy = 0;
    try:
        for i in range(len(xvec)):
            if i in xvec:
                x2 += xvec[i]*xvec[i]/(xl*xl)
            if i in yvec:
                y2 += yvec[i]*yvec[i]/(yl*yl)
                if i in xvec:
                    xy += xvec[i]*yvec[i]/(xl*yl)
    except:
        print("error at position ",i)
    if (x2+y2 == 0):
        cos = 0
    else:
        cos = xy/(math.sqrt(x2)+math.sqrt(y2))
    return cos
    
def newrandomvector(n,k):
    vec = {}
    if (k > 0 and k % 2 == 0):
        nonzeros = random.sample(list(range(n)),k)
        negatives = random.sample(nonzeros,int(k/2))
        for i in nonzeros:
            vec[i] = 1;
        for i in negatives:
            vec[i] = -1;
    return vec

In [40]:
def dowordspacefile(vocab):
    linepattern = re.compile(r'^\(\"([\-\w]+)\"\s+#S2000([\d\,\-\.\-\+;e]+)*:\s+#S2000([\d\-\+;\.e\,]+)*:\s+#S2000([\.\d\-\+;e\,]+)*:\s+(\d+)\)')
    vecpattern = re.compile(r';?(\d+)([\+\-][\,\.\d]+(e-\d\d)?)')
    alicefile=open(wordspacefile,'r')
    global vectorspace
    vectorspace={}
    global globalfrequency
    globalfrequency={}
    aliceline = alicefile.readline()
    a=1
    b=0
    w=0
    sparsevec={}
    while aliceline:
        a = a+1
        m=linepattern.match(aliceline)
        if m:
            token=m.groups()[0]
            ind=m.groups()[1]
            doc=m.groups()[2]
            ctx=m.groups()[3]
            freq=int(m.groups()[4])
            try:
                if (token in vocab):
                    cells = re.findall(vecpattern,ctx)
                    for c in cells:
                        val = c[1]
                        kop=val.translate(str.maketrans(",",".","+"))
                        try:
                            sparsevec[c[0]] = float(kop)
                        except:
                            print("weird number: ",kop)
                    vectorspace[token] = sparsevec
                    globalfrequency[token] = freq
                    sparsevec={}
                    w=w+1
            except:
                print("fail: ",token," ",a)
        else:
            b=b+1
#            print("no match for ",aliceline[0:10])
        aliceline = alicefile.readline()
    print(a," lines seen; ",w," items added ",b," items disregarded")
    alicefile.close()




In [50]:
def dotextfile():
    cleanuppattern=re.compile(r'[\.\'\!\*\?\+,;\:\-\/]+')
    utterances={}
    samplefile=open(textfile,'r')
    utterance=samplefile.readline().rstrip()
    while utterance:
        print(".",end="")
        uttrep=[]
        cleanedutterance=re.sub(cleanuppattern,"",utterance)
        words=cleanedutterance.lower().split()
        for word in words: 
            if word not in vectorspace:
                vec=newrandomvector(2000,10)
                weight=1
            else:
                vec=vectorspace[word]
                weight=float(lexicon[word])/float(globalfrequency[word])
            uttrep=sparseadd(uttrep,vec,weight)
        utterances[utterance]=uttrep
        utterance=samplefile.readline().rstrip()
    samplefile.close()
    return utterances

In [4]:
lexicon = dolexicon()
print(len(lexicon))

18699


In [47]:
print(time.ctime())
dowordspacefile(lexicon)
print(len(vectorspace))
print(time.ctime())

Tue Oct 24 20:16:59 2017
fail:  alito   276457
fail:  anti-u   299883
fail:  kph   308003
fail:  schenley   521897
531166  lines seen;  17081  items added  28811  items disregarded
17081
Tue Oct 24 20:18:00 2017


In [46]:

x=12.8507
y=5.64446486869267
print(x*y)


72.53532468810889
Tue Oct 24 20:16:25 2017


In [51]:
print(len(lexicon))
print(len(vectorspace))
print(time.ctime())
utterances=dotextfile()
print(time.ctime())
print(len(utterances))


18699
17081
Tue Oct 24 20:24:59 2017
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [55]:
kk=list(utterances.keys())
for i in range(10):
    k=random.randrange(len(kk))
    u = kk[k]
    
#r=0
#for u in utterances:
#    r=r+1
#    if r>10:
#        break
    print(i)
    print(k)
    print(u)
    ss=0
    ii=0
    vec=utterances[u]
    for xx in vec:
        ss=ss+vec[xx]
        if vec[xx] > 0:
            ii=ii+1
    print(ii," ",ss)
    max1 = 0
    max2 = 0
    max3 = 0
    s=0
    best=""
    nextbest=""
    thirdbest=""
    for v in utterances:
        if v==u:
            continue
        s=s+1;
        c=lengthweightedsparsecosine(utterances[u],len(u),utterances[v],len(v))
        if (c>max1):
            max3=max2
            max2=max1
            max1=c
            thirdbest=nextbest
            nextbest=best
            best=str(s)+" "+v
        elif (c>max2):
            max3=max2
            max2=c
            thirdbest=nextbest
            nextbest=str(s)+" "+v
        elif (c>max3):
            max3=c
            thirdbest=str(s)+" "+v
             
    print("  ",max1,"  ",best)
    print("  ",max2,"  ",nextbest)
    print("  ",max3,"  ",thirdbest)

       

0
1252
PA	The main Palestinian factions have also rejected Geneva. Some 3,000 protestors burned effigies of Abed Rabbo and Beilin as well as a Swiss flag at a  rally  in the West Bank town of Nablus on Friday.
1015   70.6101121353373
   0.0019641476150114158    119 NO	Some fans gathered around a shut-down bar behind the Place du Capitole, while  riot  police watched in tense silence.
   0.0015034440742381517    2427 NA	The woman also shouted ``Falun Gong'' and raised a yellow scarf before uniformed police snatched it away and  marched  her to a car.
   0.001460218767504648    4639 NA	Like many of the mutinous soldiers engaged in conflict with the army in Kanyabayonga, Rwibashira and Amisi are both drawn from the ranks of the former rebel Congolese  Rally  for Democracy (RCD).
1
2070
NA	Seven civilians, including a child, were among 55 people killed in a NATO air  strike  in Afghanistan that targeted fuel tankers hijacked by the Taliban, a local official said Sunday.
1021   1.6204634504

In [ ]:
print(len(lexicon))
print(len(vectorspace))
#samplefile=open(textfile,'r')
#utterance1=samplefile.readline().rstrip()
#utterance2=samplefile.readline().rstrip()
#samplefile.close()
print(utterance1)
print(utterance2)
cleanuppattern=re.compile(r'[\.\'\!\*\?\+,;\:\-\/]+')
cleanedutterance1=re.sub(cleanuppattern,"",utterance1)
words1=cleanedutterance1.lower().split()
cleanedutterance2=re.sub(cleanuppattern,"",utterance2)
words2=cleanedutterance2.lower().split()
uttrep={}
for word in words2: 
    if word not in vectorspace:
        print(word," is new")
    else:
        print(word," is known: ",vectorspace[word])
        sparseadd(uttrep,vectorspace[word],lexicon[word])

In [ ]:


a=newrandomvector(100,10)
b=newrandomvector(100,10)
print(a)
print(b)
c=sparseadd(a,b)
print(c)
print(3533268.0173778143)
print(str(3533268.0173778143))

In [ ]:
k=8
nonzeros = random.sample(list(range(10)),k)
negatives = random.sample(nonzeros,int(k/2))
print(nonzeros)
print(negatives)

hasch={}
hasch[1]="ett"
hasch[8]="åtta"
for k in hasch:
    print(k)
    


In [ ]:
grammar = {("np", "vp") : ["s"],
           ("det", "n") : ["np"],
           ("v", "np") : ["vp"],
           ("be", "prpt") : ["v"],
           ("adj", "np") : ["np"]}
oplexicon = {
           "not" : ["neg"],
           "the" : ["det"],
           "fm" : ["TMA"],
           "fp" : ["TMA"],
           "ft" : ["TMA"],
           "na" : ["TMA"],
           "nn" : ["TMA"],
           "of" : ["TMA"],
           "pa" : ["TMA"],
           "xx" : ["TMA"],
          }

def parse(words, gram):
    length=len(words)
    chart = [[lexicon.get(w, 0) for w in words]]
    for r in range(length-1):
        print("  r = ", r)
        chart.append([[] for i in range(length-r-1)])
    for row in chart:
        print(row)
    for phrase_length in range(2, length+1):
        print("phrase_length =", phrase_length)
        for left in range(0, length-phrase_length+1):
            print("  left = ", left)
            for left_length in range(1,phrase_length):
                right = left+left_length
                right_length = phrase_length-left_length
                print("    left length = ", left_length,
                      "right = ", right,
                      "right_length =", right_length)
                for left_cat in chart[left_length-1][left]:
                    for right_cat in chart[right_length-1][right]:
                        print("      left_cat = ", left_cat,
                              "      right_cat =", right_cat)
                        phrase_cat = gram.get((left_cat, right_cat), [])
                        print("      phrase_cat =", phrase_cat)
                        for c in phrase_cat:
                            cell = chart[left_length+right_length-1][left]
                            if c not in cell:
                                cell.append(c)
        for row in chart:
            print(row)
    for row in chart:
        print(row)


In [ ]:
nonzeros

In [36]:
oneutterance="This is a test of an utterance with FEW known words and a 4566 (number)."
cleanuppattern=re.compile(r'[\.\'\!\*\?\+,;\:\-\/]+')
uttrep=[]
cleanedutterance=re.sub(cleanuppattern,"",oneutterance)
print(cleanedutterance)
words=cleanedutterance.lower().split()
print(words)
for word in words: 
    print(word)
    if word not in vectorspace:
        vec=newrandomvector(2000,10)
        weight=1
    else:
        vec=vectorspace[word]
        weight=lexicon[word]
    print(weight)
    uttrep=sparseadd(uttrep,vec,weight)


This is a test of an utterance with FEW known words and a 4566 (number)
['this', 'is', 'a', 'test', 'of', 'an', 'utterance', 'with', 'few', 'known', 'words', 'and', 'a', '4566', '(number)']
this
2.64319241476306
is
2.00541778832661
a
0.590344462656673
test
6.04992997680083
of
0.540204315157007
an
2.06869190235121
utterance
1
with
1.65137404017574
few
4.3917019001973
known
4.92391871394461
words
5.9698872691273
and
0.64238458306848
a
0.590344462656673
4566
1
(number)
1


In [38]:
lexicon[0]


KeyError: 0

In [39]:
lexicon["with"]

'1.65137404017574'

In [59]:
lexicon["yacht"]

'7.14854226546894'